Work in progress - Note to myself, still to do:
    - test error calculation in feature weight function, set n as parameter, the function is time consuming)
    - check if error calculation is as time consuming in bycicle data as it is here
    - find a better way for feature reduction
    - work through Statitics for ML book for additional insights
    
    

# Function for Quick And Dirty Linear Regression Modelling on any Dataset

The preprocessing is not sophisticated at all, but 
- numerical data is normalized, 
- NaN are removed, 
- and there is One-Hot-Encoding for the categorical data.

**Note:** If the data is already centered and you want `fit_intercept = False`, then normalization will not be applied. See [here](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) for more info, and see Bycicle notebook for an example with centered data.

### Loading libraries and data

In [55]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import resample  # for error calculation of effects

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

# display all columns in df
from IPython.display import display
pd.options.display.max_columns = None

In [56]:
df = pd.read_csv('data_survey.csv')
df.sample(3)

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,WorkPayCare,KinshipDevelopers,ChallengeMyself,CompetePeers,ChangeWorld,JobSeekingStatus,HoursPerWeek,LastNewJob,AssessJobIndustry,AssessJobRole,AssessJobExp,AssessJobDept,AssessJobTech,AssessJobProjects,AssessJobCompensation,AssessJobOffice,AssessJobCommute,AssessJobRemote,AssessJobLeaders,AssessJobProfDevel,AssessJobDiversity,AssessJobProduct,AssessJobFinances,ImportantBenefits,ClickyKeys,JobProfile,ResumePrompted,LearnedHiring,ImportantHiringAlgorithms,ImportantHiringTechExp,ImportantHiringCommunication,ImportantHiringOpenSource,ImportantHiringPMExp,ImportantHiringCompanies,ImportantHiringTitles,ImportantHiringEducation,ImportantHiringRep,ImportantHiringGettingThingsDone,Currency,Overpaid,TabsSpaces,EducationImportant,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,CousinEducation,WorkStart,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,AuditoryEnvironment,Methodology,VersionControl,CheckInCode,ShipIt,OtherPeoplesCode,ProjectManagement,EnjoyDebugging,InTheZone,DifficultCommunication,CollaborateRemote,MetricAssess,EquipmentSatisfiedMonitors,EquipmentSatisfiedCPU,EquipmentSatisfiedRAM,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
3873,3874,Professional developer,No,India,"Yes, full-time",Employed full-time,Bachelor's degree,Computer programming or Web development,Never,100 to 499 employees,I prefer not to answer,2 to 3 years,1 to 2 years,NaN,Web developer,Back-end Web developer,NaN,NaN,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a hard ""g,"" like ""gift""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not very important,Self-taught,Trade book; Textbook; Stack Overflow Q&A; Stac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Waterfall; Agile,Team Foundation Server,Multiple times a day,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4608,4609,Professional developer,"Yes, I contribute to open source projects",India,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"More than half, but not all, the time",500 to 999 employees,"Privately-held limited company, not in startup...",3 to 4 years,3 to 4 years,NaN,Web developer,Full stack Web developer,NaN,NaN,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Function for one-step LinReg modelling

Function for cleaning, preparing, fitting and evaluating LinReg model:

**Attention**:
- All columns with dtype object and category will be one-hot-encoded. Make shure they all have categorical character
- All NaN in numerical cols will be imputed with mean
- All NaN in non-numerical cols are mixed in zero category, for own column change 'dummy_na' parameter)

In [57]:
def LinRegModel(df, target_col, dummy_na=False, test_size=.3, rand_state=666):
    '''
    INPUT:
    df: dataframe
    target_col: name of target column 
    dummy_na: bool holding whether to dummy NA vals of categorical columns or not (default: False)
    test_size: proportion of data to set aside in test dataset (default: 0.3)
    rand_state: random state for splitting the data into training and test sets (default: 666) 
    
    OUTPUT:
    mse_score: Mean Squarred Error of prediction
    r2_score: R squarred / variance score (1 would be perfect prediction)
    lin_model: sklearn model object
    X_train, X_test, y_train, y_test: output from sklearn train test split used for optimal model
    
    Your function should:
    1. Drop the rows with missing target values
    2. Drop columns with NaN for all the values
    3. Use create_dummy_df to dummy categorical columns
    4. Fill the mean of the column for any missing values 
    5. Split your data into an X matrix and a target vector y
    6. Create training and test sets of data
    7. Instantiate a LinearRegression model with normalized data
    8. Fit your model to the training data
    9. Predict the target for the training data and the test data
    10. Obtain an rsquared value for both the training and test data
    '''
    
    # cleaning rows with NaN in target col
    df = df.dropna(subset=[target_col], axis=0)
    # dropping columns with all NaN
    df = df.dropna(how='all', axis=1)
    
    # separate target column
    X = df.drop(target_col, axis=1)
    y = df[target_col].copy()
    
    # impute mean for missing values in num cols
    for col in X.select_dtypes(include=['float', 'int']).columns:
        X[col].fillna(df[col].mean(), inplace=True)
    
    # One-Hot-Encoding of non-numerical columns (drop original column and add encoded columns)
    for col in X.select_dtypes(include=['object', 'category']).columns:
        X = pd.concat([X.drop(col, axis=1), pd.get_dummies(X[col], prefix=col, prefix_sep='_', 
                                                             drop_first=True, dummy_na=False)], axis=1)

    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state) 

    lin_model = LinearRegression(normalize=True) # Instantiate with normalized values
    lin_model.fit(X_train, y_train) #Fit
        
    #Predict and score the model
    y_test_preds = lin_model.predict(X_test) 
    mse_score = mean_squared_error(y_test, y_test_preds)
    rsquared_score = r2_score(y_test, y_test_preds)
    
    return mse_score, rsquared_score, lin_model, X_train, X_test, y_train, y_test, X, y # X,y used for later optimization


Run the model.

**NOTE**
- RMSE has to be calculated from MSE. Both are sensitive to outliers. If you have lot's of outliers MAE might be preferable.
- R2-score can be problematic, there is also an adjusted R2-score to consider.

In [58]:
df = df
target_col = 'Salary'

mse_score, rsquared_score, lin_model, X_train, X_test, y_train, y_test, X, y= LinRegModel(df, target_col)
print("The RMSE on the test data is {:.2f}. The r2-score is {:.2f}.".format(np.sqrt(mse_score), rsquared_score))

The RMSE on the test data is 29388.70. The r2-score is 0.47.


### Feature Importance / Coefficients
***Attention: They may be completely different from those of an optimized model! See below.***

The default penalty on coefficients using linear regression in sklearn is a ridge (also known as an L2) penalty.  Because of this penalty, and because all the variables were normalized, we can look at the size of the coefficients in the model as an indication of the impact of each variable on the target variable. The larger the coefficient, the larger the expected impact. 

In [59]:
def coef_weights(model, X_train):
    '''
    INPUT:
    coefficients: the coefficients of the linear model 
    X_train: the training data, so the column names can be used
    
    OUTPUT:
    coefs_df: a dataframe holding the coefficient estimate, and abs(estimate)
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = model.coef_
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

In [64]:
def print_coef_weights(model, X=X, y=y):
    '''
    INPUT:
    coefficients: the coefficients of the linear model 
    X: the input data, so the column names can be used
    y: the target data, for resampling 
    
    OUTPUT:
    coefs_df: a dataframe holding the coefficient estimate, and abs(estimate)
    '''
    coefs_df = pd.DataFrame(index=X.columns)
    coefs_df['effect'] = model.coef_.round(1)
    
    # calculate modulo of coefs_ for sorting the df only, drop then
    coefs_df['abs_coefs'] = np.abs(coefs_df['effect'])
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    coefs_df.drop('abs_coefs', axis=1, inplace=True)
    
    # add uncertainty with help of bootstrap resamling
    np.random.seed(1)
    err = np.std([model.fit(*resample(X, y)).coef_ for i in range(1)], 0)
    coefs_df['error'] = err.round(0)
    
    
    return coefs_df

Run function:

In [61]:
coef_df = coef_weights(lin_model, X_train)
coef_df.head(20)

,est_int,coefs,abs_coefs
7380,WantWorkLanguage_Ruby; Rust; SQL,82659.722535,82659.722535
651,DeveloperType_Web developer; Mobile developer;...,76229.405626,76229.405626
4761,HaveWorkedLanguage_CoffeeScript; JavaScript; P...,71586.106038,71586.106038
4814,HaveWorkedLanguage_F#; Haskell,70544.464050,70544.464050
6434,WantWorkLanguage_Clojure; Go; Haskell; Python;...,68241.885526,68241.885526
4660,HaveWorkedLanguage_C; SQL,-68056.232735,68056.232735
237,DeveloperType_Desktop applications developer; ...,63039.835016,63039.835016
983,ImportantBenefits_Annual bonus; Health benefit...,62042.970742,62042.970742
2693,EducationTypes_Part-time/evening course; Self-...,60853.108355,60853.108355
1704,ImportantBenefits_Stock options; Annual bonus;...,57034.247034,57034.247034


In [65]:
coef_df = print_coef_weights(lin_model)
coef_df.head(20)

,effect,error
WantWorkLanguage_Ruby; Rust; SQL,82659.7,0.0
DeveloperType_Web developer; Mobile developer; Embedded applications/devices developer; Database administrator; DevOps specialist,76229.4,0.0
HaveWorkedLanguage_CoffeeScript; JavaScript; Python,71586.1,0.0
HaveWorkedLanguage_F#; Haskell,70544.5,0.0
WantWorkLanguage_Clojure; Go; Haskell; Python; Rust,68241.9,0.0
HaveWorkedLanguage_C; SQL,-68056.2,0.0
DeveloperType_Desktop applications developer; Quality assurance engineer; Other,63039.8,0.0
ImportantBenefits_Annual bonus; Health benefits; Professional development sponsorship; Remote options; Child/elder care,62043.0,0.0
EducationTypes_Part-time/evening course; Self-taught; Coding competition; Open source contributions,60853.1,0.0
ImportantBenefits_Stock options; Annual bonus; Private office; Professional development sponsorship; Remote options,57034.2,0.0


## Reduce Overfitting / Features and Optimize Model

Define function for finding an optimized model (with less overfitting thanks to a reduction in features).

**Note:**
- The decision on witch features are kept is depending on NaN, this is not good. I should be able to do better

In [ ]:
def optimizeLinModel(X, y, cutoffs, test_size = .30, random_state=42, plot=True):
    '''
    INPUT
    X: pandas dataframe, X matrix
    y: pandas dataframe, response variable
    cutoffs: list of ints, cutoff for number of non-zero values in dummy categorical vars
    test_size: float between 0 and 1, default 0.3, determines the proportion of data as test data
    random_state: int, default 42, controls random state for train_test_split
    plot: boolean, default 0.3, True to plot result

    OUTPUT
    r2_scores_test: list of floats of r2 scores on the test data
    r2_scores_train: list of floats of r2 scores on the train data
    opt_model: model object from sklearn
    X_train, X_test, y_train, y_test: output from sklearn train test split used for optimal model
    '''
    r2_scores_test, r2_scores_train, num_feats, results = [], [], [], dict()
    
    for cutoff in cutoffs:
        #reduce X matrix
        reduce_X = X.iloc[:, np.where((X.sum() > cutoff) == True)[0]]
        num_feats.append(reduce_X.shape[1])
        #split the data into train and test
        X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)
        #fit the model and obtain pred response
        lm_model = LinearRegression(normalize=True)
        lm_model.fit(X_train, y_train)
        y_test_preds = lm_model.predict(X_test)
        y_train_preds = lm_model.predict(X_train)
        #append the r2 value from the test set
        r2_scores_test.append(r2_score(y_test, y_test_preds))
        r2_scores_train.append(r2_score(y_train, y_train_preds))
        results[str(cutoff)] = r2_score(y_test, y_test_preds)

    if plot:
        plt.plot(num_feats, r2_scores_test, label="Test", alpha=.5)
        plt.plot(num_feats, r2_scores_train, label="Train", alpha=.5)
        plt.xlabel('Number of Features')
        plt.ylabel('Rsquared')
        plt.title('Rsquared by Number of Features')
        plt.legend(loc='lower right')
        plt.show()

    best_cutoff = max(results, key=results.get)

    #reduce X matrix
    reduce_X = X.iloc[:, np.where((X.sum() > int(best_cutoff)) == True)[0]]
    num_feats.append(reduce_X.shape[1])

    #split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

    #fit the model
    opt_model = LinearRegression(normalize=True)
    opt_model.fit(X_train, y_train)

    return r2_scores_test, r2_scores_train, opt_model, X_train, X_test, y_train, y_test

In [ ]:
#cutoffs here pertains to the number of missing values allowed in the used columns.
#Therefore, lower values for the cutoff provides more predictors in the model.
cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 25]

#Run this cell to pass your X and y to the model for testing
r2_scores_test, r2_scores_train, opt_model, X_train, X_test, y_train, y_test = optimizeLinModel(X, y, cutoffs)

Stats for optimized model:

In [ ]:
print(X_train.shape[1]) #Number of features
print(r2_scores_test[np.argmax(r2_scores_test)]) # The model we should implement test_r2
print(r2_scores_train[np.argmax(r2_scores_test)]) # The model we should implement train_r2

## Feature Importance / Coefficients - for optimized Model

In [ ]:
coef_df = coef_weights(opt_model, X_train)
coef_df.head(20)

***